In [2]:
import pandas as pd
import numpy as np

import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

import statsmodels.api as sm

import seaborn as sns

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

<h2> COMPAS: Correctional Offender Management Profiling for Alternative Sanctions </h2>

![imagen](https://static.propublica.org/projects/algorithmic-bias/assets/img/generated/opener-b-crop-960*540-00796e.jpg)
<b><h6> Imagen 1 - Machine Bias (ProPublica) </h6></b>

- Algoritmo utilizado en el sistema de justicia criminal de Estados Unidos para predecir la probabilidad o riesgo de reincidencia de un acusado.

- Tiene por objetivo  ayudar a los jueces a tomar decisiones más informadas sobre el riesgo de reincidencia.

Investigación hecha por ProPublica: Existe una clara diferencia en la distribución de los puntajes de riesgo según raza.

<b><h3>Links de interés: </h3></b>

- [Articulo ProPublica : Machine Bias](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing)

- [Metodología](https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm)

- [Códigos originales de ProPublica](https://github.com/propublica/compas-analysis/tree/master)

In [3]:
compas = pd.read_csv('https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv')
compas.shape

(7214, 53)

In [4]:
compas.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0


## Columnas de interés:

1. `age` - Edad del acusado. Variable numérica.
2. `age_cat` - Categoría de edad. Puede ser < 25, 25-45, >45.
3. `sex` - Sexo del acusado. Puede ser  'Male' o 'Female'.
4. `race` - Raza del acusado. Puede ser 'African-American', 'Caucasian', 'Hispanic', 'Asian', o 'Other'.
5. `c_charge_degree` - Grado del crimen. Es o bien M (Misdemeanor), F (Felony), o O (Ordinary traffic offense).
6. `priors_count` - Número de crimenes anteriores cometidos por el acusado. Variable numérica.
7. `days_b_screening_arrest` - Días entre el arresto y la fecha del cargo por el delito registrado en COMPAS.
8. `decile_score` - Puntaje COMPAS predicho por el sistema. Toma valores entre 0 y 10.
9. `score_text` - Categoria de decile_score. Puede Low (1-4), Medium (5-7), y High (8-10).
10. `is_recid` - Variable que indica si un acusado es reincidente o no. Puede tomar valores 1, 0 o -1.
11. `two_year_recid` - Variable que indica si un acusado tuvo reincidencia dentro de un periodo de dos años.
12. `c_jail_in` - Momento en que el acusado fue ingresado a la carcel.
13. `c_jail_out` - Momento en que el acusado fue liberado de la carcel.

In [ ]:
# Mantenemos las columnas de interés.

columns = ['age', 'c_charge_degree', 'race', 'age_cat',
           'score_text', 'sex', 'priors_count', 'days_b_screening_arrest',
           'decile_score', 'is_recid', 'two_year_recid', 'c_jail_in', 'c_jail_out']

compas = compas[columns]

In [ ]:
compas.head()

In [ ]:
compas.info()

En la metodología propuesta por ProPublica (revisar <b>Links de interés</b>) se presentan los siguientes criterios para realizar la limpieza de los datos:

- Si la fecha del cargo por el delito de un acusado evaluado por COMPAS no estaba dentro de los 30 días desde el momento en que la persona fue arrestada, asumimos que, debido a razones de calidad de datos, no tenemos el delito correcto.

- Según se indica en el código publicado por ProPublica, se etiquetó con ```is_recid = -1``` los casos para los cuales no se encontró el caso de COMPAS.

- En una línea similar, se eliminan las infracciones de tráfico comunes (aquellas con un grado de ```c_charge_degree``` <b>'O'</b>), que no resultarían en tiempo de prisión.

In [ ]:
clean_df = compas[((compas['days_b_screening_arrest'] <= 30) &\
                  (compas['days_b_screening_arrest'] >= -30) &\
                  (compas['is_recid'] != -1) &\
                  (compas['c_charge_degree'] != 'O'))]
clean_df.shape

In [ ]:
compas.shape

In [ ]:
print(f'Se eliminaron {compas.shape[0] - clean_df.shape[0]} regitros.')


In [ ]:
clean_df.info()

In [ ]:
age_count = clean_df['age_cat'].value_counts(normalize=True) * 100
race_count = clean_df['race'].value_counts(normalize=True) * 100
sex_count = clean_df['sex'].value_counts(normalize=True) * 100

## Análisis exploratorio

### Distribución de la edad

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
ax = ax.flatten()

sns.histplot(data=clean_df, x='age', color='darkblue', ax=ax[0])
sns.countplot(data=clean_df, x='age_cat', ax=ax[1])

ax[0].set_title('Distribución de la edad')
ax[1].set_title('Nº registros por categoría de edad')
plt.show()

In [ ]:
age_count

#### Interseccionalidad (edad)

In [ ]:
pd.crosstab(values=clean_df['score_text'], columns=clean_df['race'], index=clean_df['age_cat'], aggfunc=lambda x: len(x),normalize='all') * 100

: 

### Distribución de la raza

In [ ]:
fig, ax = plt.subplots(1, figsize=(8, 5))
sns.countplot(x=clean_df['race'])
plt.title('Nº registros por raza')
plt.show()